In [1]:
import mysql.connector
from mysql.connector import errorcode

import pandas as pd
from datetime import date, datetime, timedelta
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio
pio.renderers.default = 'notebook'

import random

In [2]:
cnx = mysql.connector.connect(
    user="quera_data_2",
    host="127.0.0.1", 
    password="P@ssword1401",
)

In [3]:


DB_NAME = 'football'
cursor = cnx.cursor()


In [4]:
cursor.execute(f"use {DB_NAME}")

In [5]:
query = """
SELECT position,average_score,t.id as team_id,t.Name as team_name from (
SELECT p.Positon_part1 AS position, s.Team_id AS team_id, AVG(ps.Overall_score) AS average_score
FROM playerstats AS ps
INNER JOIN player AS p ON ps.Player_id = p.id
INNER JOIN squad AS s ON p.id = s.Player_id AND ps.Season = s.Season
WHERE s.Season = 2021
GROUP BY team_id,position)
    as T1
inner join team t on team_id = t.id;
"""

cursor.execute(query)

# Fetch all data and store in a Pandas dataframe
data = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
data


,position,average_score,team_id,team_name
0,defender,43.166559,631,Chelsea FC
1,attack,41.111624,398,SS Lazio
2,midfield,59.230928,281,Manchester City
3,defender,43.532921,368,Sevilla FC
4,attack,42.066932,12,AS Roma
...,...,...,...,...
387,goalkeeper,36.208333,1108,Deportivo Alavés
388,goalkeeper,34.625000,331,CA Osasuna
389,goalkeeper,45.583333,368,Sevilla FC
390,goalkeeper,41.666667,3368,Levante UD


In [6]:
attack_data = data[data["position"]=="attack"]
attack_data = attack_data.set_index("team_id").rename({"average_score":"attack_score"},axis=1).drop("position",axis=1)
attack_data

,attack_score,team_name
team_id,,
398,41.111624,SS Lazio
12,42.066932,AS Roma
631,52.116953,Chelsea FC
131,48.170575,FC Barcelona
281,59.808226,Manchester City
...,...,...
276,34.619075,Hellas Verona
16795,35.831088,Granada CF
331,35.952502,CA Osasuna


In [7]:
defender_data = data[data["position"]=="defender"]
defender_data = defender_data.set_index("team_id").rename({"average_score":"defender_score"},axis=1).drop(["position","team_name"],axis=1)
defender_data

,defender_score
team_id,
631,43.166559
368,43.532921
46,55.580379
5,52.234647
379,38.087679
...,...
44,35.571526
749,28.898761
714,34.007110


In [8]:
merged_data = pd.concat([attack_data,defender_data], axis=1, join='inner').reset_index()
merged_data

,team_id,attack_score,team_name,defender_score
0,398,41.111624,SS Lazio,41.666620
1,12,42.066932,AS Roma,44.814795
2,631,52.116953,Chelsea FC,43.166559
3,131,48.170575,FC Barcelona,44.533915
4,281,59.808226,Manchester City,53.939977
...,...,...,...,...
93,276,34.619075,Hellas Verona,39.344243
94,16795,35.831088,Granada CF,33.758176
95,331,35.952502,CA Osasuna,33.457566
96,621,40.942679,Athletic Bilbao,41.673869


In [9]:
merged_data["need_for_attacks"] = merged_data["defender_score"] - merged_data["attack_score"]
merged_data[merged_data["need_for_attacks"]>0].sort_values("need_for_attacks",ascending=False)

,team_id,attack_score,team_name,defender_score,need_for_attacks
13,46,45.513458,Inter Milan,55.580379,10.066921
93,276,34.619075,Hellas Verona,39.344243,4.725168
68,667,40.693147,RC Strasbourg Alsace,45.018703,4.325556
76,1390,30.203203,Cagliari Calcio,33.468743,3.265541
11,681,42.634760,Real Sociedad,45.813259,3.178499
1,12,42.066932,AS Roma,44.814795,2.747863
66,273,44.191949,Stade Rennais FC,46.532692,2.340743
72,607,27.284264,Venezia FC,29.604269,2.320005
8,418,51.507127,Real Madrid,53.173393,1.666266
44,40,33.864894,FC Girondins Bordeaux,35.517461,1.652567
